In [3]:
import cv2
import numpy as np
import os
from tensorflow import keras
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import csv
import pandas as pd
%load_ext tensorboard

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

#랜드마크 그려주기
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #POSE 랜드마크
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #왼손 랜드마크
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #오른손 랜드마크

def extract_keypoints_pose(result):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    return np.concatenate([pose])

def extract_keypoints_hand(result):
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([lh, rh])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# 오른 손 65y좌표, 오른 어깨 49

In [ ]:
cap = cv2.VideoCapture(0)

i = 0
#Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # feed 읽기
        ret, frame = cap.read()
        
        #detections 만들기
        image, results = mediapipe_detection(frame, holistic)
        
        #랜드마크 그리기
        draw_landmarks(image,results)
        if i%30 ==0:
            keypoint = extract_keypoints_pose(results)
            print(keypoint[65],keypoint[49],keypoint[61],keypoint[45])
            if keypoint[65]<0.8 and keypoint[61]<0.8:
                print("hand_up")
            elif keypoint[65]<0.8:
                print("right_hand_up")
            elif keypoint[61]<0.8:
                print("left_hand_up")
            
        #화면에 보여주기
        cv2.imshow("OpenCV Feed",image)
        i += 1
        #화면 종료
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1.662654161453247 0.5904155373573303 1.381731629371643 0.6779868602752686
0.973514974117279 0.34214258193969727 1.020891547203064 0.3863603174686432
0.9215455651283264 0.35841092467308044 0.9255971908569336 0.38409623503685
0.8730152249336243 0.3891821801662445 0.8837199807167053 0.4004228413105011
0.7894724607467651 0.3745478689670563 0.7867604494094849 0.39728453755378723
hand_up
0.7285807132720947 0.3757840394973755 0.7867773175239563 0.3917831778526306
hand_up
0.7210670113563538 0.37985554337501526 0.7224101424217224 0.39029660820961
hand_up
0.7226210236549377 0.3764875531196594 0.8819219470024109 0.3963669538497925
right_hand_up
0.8730165958404541 0.3782421052455902 0.8879870772361755 0.40036535263061523
0.8667680025100708 0.3804735243320465 0.7872317433357239 0.4004860818386078
left_hand_up
0.8704490065574646 0.3859878182411194 0.6394276022911072 0.4007382094860077
left_hand_up
1.4189083576202393 0.7005403637886047 1.0676891803741455 0.6124204993247986
0.895071804523468 0.3702114

In [6]:
len(keypoint)

132